### Import en samen

In [122]:
import numpy
written_train =numpy.load("written_train(1).npy")
spoken_train =numpy.load("spoken_train(1).npy")
match_train = numpy.load("match_train(1).npy")
feature_amount = spoken_train[0].shape[1]
print("shape written:", written_train.shape)
print("shape spoken", spoken_train.shape)
print("shape spoken indiv:", spoken_train[4].shape)
print(feature_amount)
print(match_train.shape)



shape written: (45000, 784)
shape spoken (45000,)
shape spoken indiv: (38, 13)
13
(45000,)


In [123]:
import pandas as pd
from numpy import array

maxs = []
for i in range(0, len(spoken_train)):
    maxs.append(spoken_train[i].shape[0])
maxlen_spoken_train = max(maxs)

from keras.preprocessing.sequence import pad_sequences

spoken_train_3d= pad_sequences(spoken_train, maxlen= max(maxs))
print(spoken_train_3d.shape)

spoken_train_2d =spoken_train_3d.reshape(45000, maxlen_spoken_train*feature_amount)

spoken_train_2d 




(45000, 93, 13)


array([[ 0,  0,  0, ..., -1,  0,  0],
       [ 0,  0,  0, ..., -2,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       ...,
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ..., -1,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0]], dtype=int32)

### Scaling

In [124]:
#Standardscaler
from sklearn.preprocessing import StandardScaler

SS_scaler = StandardScaler()
spoken_train_SSscaler = SS_scaler.fit_transform(spoken_train_2d)
written_train_SSscaler = SS_scaler.fit_transform(written_train)

Scaler = "Standard scaler"

#When using standard scaler
X_values = numpy.hstack([spoken_train_SSscaler, written_train_SSscaler])
print(X_values.shape)
X_valSet = "X_val set for standard scaler"

(45000, 1993)


In [125]:
#split
from sklearn.model_selection import train_test_split
X_values, X_acctest, match_train, y_acctest = train_test_split(X_values, match_train, test_size=1/3, random_state=999)
X_train, X_test, y_train, y_test = train_test_split(X_values, match_train, test_size=0.2, random_state=123)
c,d = X_test.shape

# CAUTION

The data is first splitted into 'data for the model' and 'data to check'(acctest). The data for the model is then again splitted in train and test data. Afterwords when you get a good score you can test it on the check data (acctest) as if it were the data we had to submit in order to see how well it actually performs.

### Oversample

In [126]:
from imblearn.over_sampling import SMOTE



print("Before OverSampling, counts of label '1': {}".format(sum(y_train==0)))
print("Before OverSampling, counts of label '2': {} ".format(sum(y_train==1)))

sm = SMOTE(random_state=4)
X_res, y_res = sm.fit_sample(X_train, y_train.ravel())

print('After OverSampling, the shape of X: {}'.format(X_res.shape))
print('After OverSampling, the shape of y: {} \n'.format(y_res.shape))

print("After OverSampling, counts of label '1': {}".format(sum(y_res==0)))
print("After OverSampling, counts of label '2': {}".format(sum(y_res==1)))


X= numpy.array(X_res)
y= numpy.array(y_res)

X_train=X
y_train=y
print(y_train.shape)

a, b = X_train.shape


Before OverSampling, counts of label '1': 21614
Before OverSampling, counts of label '2': 2386 
After OverSampling, the shape of X: (43228, 1993)
After OverSampling, the shape of y: (43228,) 

After OverSampling, counts of label '1': 21614
After OverSampling, counts of label '2': 21614
(43228,)


### try 1

In [91]:
X_train = X_train.reshape(a,b,1)
X_test = X_test.reshape(c,d,1)


In [92]:
from keras.utils import to_categorical
y_train = to_categorical(y_train)

In [106]:
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten, MaxPool1D
#create model
model = Sequential()
#add model layers
model.add(Conv1D(64, kernel_size=2, activation='relu', input_dim=b))
model.add(Conv1D(32, kernel_size=2, activation='relu'))
model.add(MaxPool1D())
#model.add(Flatten())
model.add(Dense(1, activation='softmax'))

/Users/Janne/tutorial-env/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, kernel_size=2, activation="relu", input_shape=(None, 199...)`
  


In [94]:
#y_train = y_train.reshape(a,1)


In [107]:
model.compile(
    optimizer='adam', 
    loss='categorical_crossentropy', 
    metrics=['accuracy'])

hist = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3)


ValueError: Error when checking input: expected conv1d_54_input to have 3 dimensions, but got array with shape (43228, 1993)

In [ ]:
from sklearn.metrics import accuracy_score

y_pred = model.predict_classes(X_testecht, verbose=1) #moet X_test zijn!

print(y_pred)

print(1 in y_pred)
y_pred = y_pred.reshape(15000,)
print(y_pred)

### try 2

In [129]:
k, = y_train.shape
y_train = y_train.reshape((k,1))
f, = y_test.shape
y_test = y_test.reshape((f, 1))

X_train = X_train.reshape(a,b,1)
X_test = X_test.reshape(c,d,1)

In [128]:
k

(43228,)

In [130]:
from keras.models import Sequential
from keras.layers import Dense, Activation

model = Sequential()
model.add(Dense(32, input_dim=1993))
model.add(Activation('relu'))

In [132]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalAveragePooling1D, MaxPooling1D

model = Sequential()
model.add(Conv1D(64, 3, activation='relu', input_shape=(1993, 1)))
model.add(Conv1D(64, 3, activation='relu'))
model.add(MaxPooling1D(3))
model.add(Conv1D(128, 3, activation='relu'))
model.add(Conv1D(128, 3, activation='relu'))
model.add(GlobalAveragePooling1D())
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.fit(X_train, y_train, batch_size=16, epochs=10)
score = model.evaluate(X_test, y_test, batch_size=16)

Epoch 1/10
43228/43228 [==============================] - 316s 7ms/step - loss: 0.5519 - acc: 0.7183
Epoch 2/10
43228/43228 [==============================] - 317s 7ms/step - loss: 0.4082 - acc: 0.8228
Epoch 3/10
43228/43228 [==============================] - 302s 7ms/step - loss: 0.3487 - acc: 0.8612
Epoch 4/10
43228/43228 [==============================] - 302s 7ms/step - loss: 0.3246 - acc: 0.8780
Epoch 5/10
43228/43228 [==============================] - 302s 7ms/step - loss: 0.3088 - acc: 0.8853
Epoch 6/10
43228/43228 [==============================] - 296s 7ms/step - loss: 0.2978 - acc: 0.8926
Epoch 7/10
43228/43228 [==============================] - 296s 7ms/step - loss: 0.2851 - acc: 0.8993
Epoch 8/10
43228/43228 [==============================] - 297s 7ms/step - loss: 0.2730 - acc: 0.9049
Epoch 9/10
43228/43228 [==============================] - 296s 7ms/step - loss: 0.2712 - acc: 0.9068
Epoch 10/10
6000/6000 [==============================] - 12s 2ms/step


### Testdata

In [133]:
written_test = numpy.load("written_test(1).npy")
spoken_test =  numpy.load("spoken_test(1).npy")

In [134]:
import pandas as pd
from numpy import array

maxs = []
for i in range(0, len(spoken_test)):
    maxs.append(spoken_test[i].shape[0])
maxlen_spoken_test = max(maxs)

from keras.preprocessing.sequence import pad_sequences

spoken_test_3d= pad_sequences(spoken_test, maxlen= max(maxs))
print(spoken_test_3d.shape)

spoken_test_2d =spoken_test_3d.reshape(15000, maxlen_spoken_test*feature_amount)

spoken_test_2d

(15000, 93, 13)


array([[ 0,  0,  0, ..., -1,  0,  1],
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ..., -2,  0,  1],
       ...,
       [ 0,  0,  0, ..., -2, -1,  0],
       [ 0,  0,  0, ..., -2, -1,  0],
       [ 0,  0,  0, ...,  0,  0,  0]], dtype=int32)

In [135]:
#Standardscaler
from sklearn.preprocessing import StandardScaler

SS_scaler = StandardScaler()
spoken_test_SSscaler = SS_scaler.fit_transform(spoken_test_2d)
written_test_SSscaler = SS_scaler.fit_transform(written_test)

Scaler = "Standard scaler"

#When using standard scaler
X_testecht = numpy.hstack([spoken_test_SSscaler, written_test_SSscaler])



In [138]:
X_acctest = X_acctest.reshape(15000,1993,1)

### Acc score

In [139]:
from sklearn.metrics import accuracy_score

y_pred = model.predict(X_acctest, verbose=1) #moet X_test zijn!

print(y_pred)

print(1 in y_pred)
y_pred = y_pred.reshape(15000,)
print(y_pred)

15000/15000 [==============================] - 29s 2ms/step
[[0.11533731]
 [0.3047256 ]
 [0.1788943 ]
 ...
 [0.08398203]
 [0.13376065]
 [0.20282027]]
False
[0.11533731 0.3047256  0.1788943  ... 0.08398203 0.13376065 0.20282027]


In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_pred, y_val)
